In [1]:
from glob import glob
import re
import mne_bids
import numpy as np
import pandas as pd
from collections import defaultdict
from mne.parallel import parallel_func

import mvEEG
import matplotlib.pyplot as plt
import tqdm as tqdm
%load_ext autoreload
%autoreload 2

c:\Users\Darius\.conda\envs\mvload\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
ROOT = r"E:\datasets\supracapacity\derivatives\\"
DATA_DIR = ROOT + "pypeline"

EXPERIMENT_NAME='supracapacity'

dropped_chans={
    'eeg':['Fp1','Fp2'],
    'eog':'ALL',
    'eyegaze':'ALL',
    'pupil':'ALL',
    'misc':'ALL'
}

conditions = ['memory/ss2/no_placeholders','memory/ss2/placeholders',
                'memory/ss4/no_placeholders','memory/ss4/placeholders',
                'memory/ss6/no_placeholders','memory/ss6/placeholders',
                'memory/ss8/no_placeholders','memory/ss8/placeholders',
                'attention/ss2/no_placeholders','attention/ss2/placeholders',
                'attention/ss4/no_placeholders','attention/ss4/placeholders',
                'attention/ss6/no_placeholders','attention/ss6/placeholders',
                'attention/ss8/no_placeholders','attention/ss8/placeholders']

cr = mvEEG.Crossnobis()
wrangl = mvEEG.Wrangler(data_dir=DATA_DIR,experiment_name=EXPERIMENT_NAME,dropped_chans=dropped_chans,conditions=conditions,n_folds=1000)
RUN_DESCRIPTION="RSA"


In [5]:

for isub in tqdm.tqdm(range(wrangl.nsub)):
    
    try:
        Xdata,ydata = wrangl.load_eeg(isub,reject=False)
    except:
        continue


    sub_path = mne_bids.BIDSPath(root=ROOT + 'mvEEG',task=EXPERIMENT_NAME,subject=wrangl.subs[isub],description=RUN_DESCRIPTION,extension='.npy',check=False)
    sub_path.mkdir()

    parallel,p_func,_ = parallel_func(cr.crossnobis_across_time,n_jobs = -1)

    out = parallel(
        p_func(
            X_train=X_train,
            X_test=X_test,
            y_train=y_train,
            y_test=y_test,
        )
        for X_train,X_test,y_train,y_test in wrangl.bin_and_split(Xdata,ydata,test_size=0.5)
    )
    rdm = np.stack(out)

    sub_path.update(suffix='RDM')
    np.save(sub_path.fpath,rdm)
    sub_path.update(suffix='rdm_times')
    np.save(sub_path.fpath,wrangl.t)




  3%|▎         | 1/31 [00:47<23:59, 47.99s/it]


KeyboardInterrupt: 